<a href="https://colab.research.google.com/github/gabrielfea/EstudokNN-/blob/main/Projeto3_Gabriel_Freitas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Projeto 3 - DISCIPLINA DE RECONHECIMENTO DE PADRÕES - MESTRADO UDESC**

GABRIEL VINICIUS BOIN FREITAS

Neste projeto vocês vão usar os classificadores Knn do projeto 2. Seu trabalho consiste em avaliar a
opinião dos descritores em conjunto:
Seja g(x) a opinião dos classificadores Knn (K=1,3,5,7,...) sobre uma amostra x. A classe escolhida
será aquela que obtiver maior votação. Caso não haja consenso (duas ou mais classes com mesma
votação), escolha aleatoriamente uma das mais votadas. Utilize um número ímpar de classificadores
para realizar a combinação (d=5,7,9,13,...).
– Verifique a performance do "novo classificador" combinado comparando com cada um
separadamente.
– Faça a comparação utilizando pelo menos 3 (três) bases diferentes.

#Download das bases escolhidas:

In [1]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset / escolhi somente bases com features numéricas para facilitar e não ter que tratar as categóricas
iris = fetch_ucirepo(id=53)
rice_cammeo_and_osmancik = fetch_ucirepo(id=545)
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)
letter_recognition = fetch_ucirepo(id=59)
glass_identification = fetch_ucirepo(id=42)
optical_recognition_of_handwritten_digits = fetch_ucirepo(id=80)
wine_quality = fetch_ucirepo(id=186)
sepsis_survival_minimal_clinical_records = fetch_ucirepo(id=827)

datasets = [iris,rice_cammeo_and_osmancik,breast_cancer_wisconsin_diagnostic,letter_recognition,glass_identification,optical_recognition_of_handwritten_digits,wine_quality,sepsis_survival_minimal_clinical_records]
names = ["Iris","Rice Cammeoa and Osmancik","Breast cancer Wisconsin diagnostic","Letter Recognition","Glass identification","Optical recognition of handwritten digits","Wine quality","Sepsis survival minimal clinical records"]

# Função de Split em z1 (train), z2 (test) e z3 (validation) - respeitando a quantidade por classe - parâmetro stratify:

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

def split(df, silent):
  perc_train = 0.25
  perc_test = 0.25
  perc_val = 0.50

  #padroniza com min_max_scaler
  X = df.data.features
  X = X.values
  min_max_scaler = preprocessing.MinMaxScaler()
  X_scaled = min_max_scaler.fit_transform(X)
  X = pd.DataFrame(X_scaled)

  #transforma y em categorias
  y = df.data.targets

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=perc_test, stratify=y)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=perc_val/(1-perc_test), stratify=y_train)
  z1 = pd.concat([X_train, y_train], axis=1)
  z2 = pd.concat([X_test, y_test], axis=1)
  z3 = pd.concat([X_val, y_val], axis=1)
  distribuicao = {"z1":len(z1), "z2": len(z2), "z3": len(z3)}
  if silent == False:
    print(f"Train: {perc_train*100}% |Test: {perc_test*100}% |Validation: {perc_val*100}%")
    print(distribuicao)
  return z1, z2 , z3


# Funções que determinam Ks:

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
def encontra_k(z1, z2,silent):
  k_min = 1
  k_max = len(z1) if len(z1) <= 20 else 20
  k_interval = 2
  dominio = range(k_min, k_max, k_interval)
  X = z1.iloc[:,:-1]
  y = z1.iloc[:,-1]
  X_hat = z2.iloc[:,:-1]
  y_hat = z2.iloc[:,-1]
  melhor_k = {}
  for i in dominio:
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X, y)
    melhor_k[i] = accuracy_score(y_hat, neigh.predict(X_hat))
  melhor_k = sorted(melhor_k.items(), key=lambda x:x[1])
  if silent == False:
    print(f"O melhor k foi {melhor_k[-1][0]}, que obteve acurácia de {np.round(100*melhor_k[-1][1],2)}%.")
  k = melhor_k[-1][0]
  return k

In [5]:
def teste_projeto_3(k, num_votantes=5):
  k_base = k
  lista_k = [v*3+k_base for v in range(0,num_votantes)]
  print(f"{num_votantes} ks:{lista_k} para criar os votos.")
  return lista_k

# Função que apura os votos e a votação majoritária de qual classe pertence cada amostra

In [6]:
import numpy as np
def preve_amostras_pelos_modelos(z1,z3,lista_k,silent):
  X = z1.iloc[:,:-1]
  y = z1.iloc[:,-1]
  X_hat = z3.iloc[:,:-1]
  y_hat = z3.iloc[:,-1]

  compara = pd.DataFrame()
  compara.index = X_hat.index


  for i,k in enumerate(lista_k):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X, y)
    compara[f"v{i+1}"] = neigh.predict(X_hat)
  compara2 = compara.transpose()
  compara2 = compara2.mode()
  compara2 = compara2.transpose()
  compara = pd.concat([compara, compara2], axis=1)
  compara["ground-truth"] = y_hat
  return compara

# Execução e resultados do Projeto 3:

In [21]:
for i,n in zip(datasets,names):
    silent = False
    if silent == False:
      print("-------------------")
      print("-------------------")
      print(f"\033[1m{n}\033[0m")
    z1, z2 , z3 = split(i,silent)
    k=encontra_k(z1, z2,silent)
    lista_k = teste_projeto_3(k, num_votantes=5)
    compara = preve_amostras_pelos_modelos(z1,z3,lista_k,silent)
    compara = compara.dropna(axis=1)
    #print(compara)
    for i in range(1,len(lista_k)+1):
      acuracia = np.round(100*accuracy_score(compara.iloc[:,i-1], compara["ground-truth"]),2)
      print(f"Acurácia modelo{i}: {acuracia}")
    v_maj = np.round(100*accuracy_score(compara.iloc[:,-2], compara["ground-truth"]),2)
    print(f"Acurácia Fusão: Voto majoritário: {v_maj}")


-------------------
-------------------
Iris
Train: 25.0% |Test: 25.0% |Validation: 50.0%
{'z1': 37, 'z2': 38, 'z3': 75}
O melhor k foi 3, que obteve acurácia de 97.37%.
5 ks:[3, 6, 9, 12, 15] para criar os votos.
Acurácia modelo1: 96.0
Acurácia modelo2: 96.0
Acurácia modelo3: 93.33
Acurácia modelo4: 92.0
Acurácia modelo5: 92.0
Acurácia Fusão: Voto majoritário: 93.33
-------------------
-------------------
Rice Cammeoa and Osmancik
Train: 25.0% |Test: 25.0% |Validation: 50.0%
{'z1': 952, 'z2': 953, 'z3': 1905}
O melhor k foi 15, que obteve acurácia de 91.71%.
5 ks:[15, 18, 21, 24, 27] para criar os votos.
Acurácia modelo1: 93.02
Acurácia modelo2: 93.12
Acurácia modelo3: 93.28
Acurácia modelo4: 93.18
Acurácia modelo5: 93.12
Acurácia Fusão: Voto majoritário: 93.12
-------------------
-------------------
Breast cancer Wisconsin diagnostic
Train: 25.0% |Test: 25.0% |Validation: 50.0%
{'z1': 142, 'z2': 143, 'z3': 284}
O melhor k foi 15, que obteve acurácia de 96.5%.
5 ks:[15, 18, 21, 24, 27